<a href="https://colab.research.google.com/github/mmmacedo/machine-learning/blob/main/Libras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

interpretador de Alfabeto em libras

Imports

In [1]:
from keras.utils import to_categorical, plot_model 
from keras.optimizers import SGD, Adam
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import LeakyReLU

import tensorflow as tf
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import datetime
import h5py
import time

import os

Constantes

In [2]:
EPOCHS = 30
CLASS = 21

Methodos auxiliares

In [3]:
def getDateStr():
  return str('{date:%Y%m%d_%H%M}').format(date=datetime.datetime.now())


def getTimeMin(start, end):
  return (end - start)/60


def build(width, height, channels, classes):
  """
  :param width: Largura em pixel da imagem.
  :param height: Altura em pixel da imagem.
  :param channels: Quantidade de canais da imagem.
  :param classes: Quantidade de classes para o output.
  """
  inputShape = (height, width, channels)
  model = Sequential()
  model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", input_shape=inputShape))
  model.add(LeakyReLU(alpha=0.1))
  model.add(MaxPooling2D((2, 2)))

  model.add(Conv2D(filters=32, kernel_size=(3, 3)))
  model.add(LeakyReLU(alpha=0.1))
  model.add(MaxPooling2D((2, 2)))

  model.add(Conv2D(filters=64, kernel_size=(3, 3)))
  model.add(LeakyReLU(alpha=0.1))
  model.add(MaxPooling2D((2, 2)))
  
  model.add(Flatten())
  model.add(Dense(256, activation="relu"))
  model.add(Dropout(0.5))
  model.add(Dense(classes, activation="softmax"))

  return model


Download Dataset

In [4]:
print('Download dataset')
train_datagen = ImageDataGenerator( #gera novas imagens com base nas imagens de modelo
        rescale=1./255, #normalizar cores
        shear_range=0.2, #rotacao com translacao imagem
        zoom_range=0.2, #zoom
        horizontal_flip=True, #espelhamento
        validation_split=0.25) #valida treinamento durante o mesmo

test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

_URL = 'https://drive.google.com/u/0/uc?id=1FG4qY25JD3SZq039AKr9zNreJS_gKc5I&export=download&confirm=t&uuid=e8e51c25-b974-44b7-b6c5-41431c685469&at=ALAFpqyB8WTeWslFKyZrUz-19u6p:1666460990795'
zip_dir = tf.keras.utils.get_file('libras_filtered.zip', origin=_URL, extract=True)

zip_dir_base = os.path.dirname(zip_dir)
!find $zip_dir_base -type d -print

Download dataset
303690709/303690709 [==============================] - 4s 0us/step
/root/.keras/datasets
/root/.keras/datasets/libras_filtered
/root/.keras/datasets/libras_filtered/train
/root/.keras/datasets/libras_filtered/train/M
/root/.keras/datasets/libras_filtered/train/V
/root/.keras/datasets/libras_filtered/train/B
/root/.keras/datasets/libras_filtered/train/O
/root/.keras/datasets/libras_filtered/train/F
/root/.keras/datasets/libras_filtered/train/E
/root/.keras/datasets/libras_filtered/train/R
/root/.keras/datasets/libras_filtered/train/U
/root/.keras/datasets/libras_filtered/train/N
/root/.keras/datasets/libras_filtered/train/P
/root/.keras/datasets/libras_filtered/train/T
/root/.keras/datasets/libras_filtered/train/Q
/root/.keras/datasets/libras_filtered/train/I
/root/.keras/datasets/libras_filtered/train/G
/root/.keras/datasets/libras_filtered/train/D
/root/.keras/datasets/libras_filtered/train/A
/root/.keras/datasets/libras_filtered/train/W
/root/.keras/datasets/libras_f

Cria os caminhos para as bases

In [5]:
base_dir = os.path.join(os.path.dirname(zip_dir), 'libras_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

Configura o treinamento

In [6]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True, 
        validation_split=0.25)

test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

training_set = train_datagen.flow_from_directory(
        train_dir,
        target_size=(64, 64),
        color_mode = 'rgb',
        batch_size=32,
        shuffle=False,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(64, 64),
        color_mode = 'rgb',
        batch_size=32,
        shuffle=False,
        class_mode='categorical')

early_stopping_monitor = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)

model = build(64, 64, 3, CLASS)
model.compile(optimizer=SGD(0.01), loss="categorical_crossentropy", metrics=["accuracy"])

Found 34714 images belonging to 21 classes.
Found 11548 images belonging to 21 classes.


Iniciar treinamento

In [ ]:
start = time.time()

classifier = model.fit_generator(
        training_set,
        steps_per_epoch=(training_set.n // training_set.batch_size),
        epochs=EPOCHS,
        validation_data = test_set,
        validation_steps= (test_set.n // test_set.batch_size),
        shuffle = False,
        verbose=2,
        callbacks = [early_stopping_monitor]
      )

EPOCHS = len(classifier.history["loss"])
end = time.time()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/30


Salvando o modelo criado

In [ ]:
model.save(os.path.join(os.path.dirname(zip_dir), 'model')+'/model.h5')
!find /root/.keras/datasets/model -type f -print